#### Environment Setup

In [81]:
from __future__ import division
import re
from pandas_profiling import ProfileReport
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import preprocessing
from sklearn.model_selection import validation_curve, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pickle

#### Set Parameters

In [82]:
Training_rand = "2018"
Scoring_rand  = "2019"

#### Utility Functions

In [83]:
# Null Analysis Function
def getPctMissing(series):
    num = series.isnull().sum(axis=0)
    den = len(series)
    return round(100*(num/den),2)

# drop column function
def drop_unwanted_columns(dataset,drop_list):
    dataset = dataset.drop(drop_list, axis =1)
    return dataset

def column_format_newline(col_list):
    new_col_list = list()
    for i in range(0,len(col_list)):
        col_name = col_list[i]
        col_name = col_name.replace("\n","_")
        new_col_list.append(col_name.replace(" ","_"))
    return new_col_list
    
    

#### Data Ingestion

In [84]:
PF_excel = pd.read_excel('InputDatasets/Jalaj Patidar - Data_with_Labels_new.xlsx', 'Table 1. PF Hospitals',skiprows = 9)
rand_hcris = pd.read_csv("InputDatasets/rand_hcris_cy_hosp_a_2021_02_01.csv",encoding = "ISO-8859-1")
IPPS_by_state18 = pd.read_csv('InputDatasets/MEDICARE_CHARGE_INPATIENT_DRGALL_DRG_BY STATE_FY2018.CSV')
IPPS_by_national18 = pd.read_csv('InputDatasets/MEDICARE_CHARGE_INPATIENT_DRGALL_DRG_NATIONAL_FY2018.CSV')
IPPS_by_prvdr18 = pd.read_csv('InputDatasets/MEDICARE_PROVIDER_CHARGE_INPATIENT_DRGALL_FY2018.CSV')
OPPS_by_prvdr18 = pd.read_excel('InputDatasets/MUP_OHP_R20_P04_V10_D18_APC_Provider.xlsx', 'Provider APC',skiprows = 5)
OPPS_by_state18 = pd.read_excel('InputDatasets/MUP_OHP_R20_P04_V10_D18_APC_Summary.xlsx', 'State APC',skiprows = 5)
OPPS_by_national18 = pd.read_excel('InputDatasets/MUP_OHP_R20_P04_V10_D18_APC_Summary.xlsx', 'National APC',skiprows = 5)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

rand_col_names = list()

PF_excel.columns = column_format_newline(PF_excel.columns)
PF_excel.replace(['^\s+$'], np.nan, regex = True, inplace = True)

# Colnames formatting for rand_hcris
for i in range(0,len(rand_hcris.columns)):
    pattern = "\[(.*?)\]"
    colname_list = re.findall(pattern, rand_hcris.columns[i], flags=0)
    if(len(colname_list)>1):
        rand_col_names.append(colname_list[len(colname_list)-1])
    elif(len(colname_list)==0):
        pattern = "\[(.*)"
        col = re.findall(pattern, rand_hcris.columns[i], flags=0)
        rand_col_names.append(col[0])
    else:
        rand_col_names.append(colname_list[0])
rand_hcris.replace(['^\s+$'], np.nan, regex = True, inplace = True)
rand_hcris.columns = rand_col_names

In [85]:
print(PF_excel.shape)
display(PF_excel.head())
print(rand_hcris.shape)
display(rand_hcris.head())


(3112, 29)


,Medicare_provider_number,Hospital_name,Street_address,City,State,Zip_code,"Hospital_system_or,_if_independent,_IPPS/CAH",Is_hospital_a_critical_access_hospital_(Y/N)?,"Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)",Number_of_outpatient_services,Total_private_allowed_amount_for_outpatient_services_($_millions),Simulated_Medicare_allowed_amount_for_outpatient_services_($_millions),Relative_price_for_outpatient_services,Standardized_price_per_outpatient_service,Number_of_inpatient_stays,Total_private_allowed_amount_for_inpatient_services_($_millions),Simulated_Medicare_allowed_amount_for_inpatient_services_($_millions),Relative_price_for_inpatient_services,Standardized_price_per_inpatient_stay,Total_private_allowed_amount_for_inpatient_and_outpatient_services_($_millions),Simulated_Medicare_allowed_amount_for_inpatient_and_outpatient_services_($_millions),Relative_price_for_professional_inpatient_and_outpatient_services,Relative_price_for_inpatient_facility_services,Relative_price_for_outpatient_facility_services,Relative_price_for_inpatient_and_outpatient_services,Total_private_allowed_amount_for_facility_inpatient_and_outpatient_services_($_millions),Simulated_Medicare_allowed_amount_for_facility_inpatient_and_outpatient_services_($_millions),Total_private_allowed_amount_for_professional_inpatient_and_outpatient_services_($_millions),Simulated_Medicare_allowed_amount_for_professional_inpatient_and_outpatient_services_($_millions)
0,10001,Southeast Alabama Medical\nCenter,1108 Ross Clark Circle,Dothan,AL,36301,Southeast Health,N,2.0,54.0,0.11,0.09,122.0,92.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110%,NaN,NaN,NaN,NaN,NaN
1,10005,Marshall Medical Center -\nSouth,2505 U S Highway 431\nNorth,Boaz,AL,35957,Marshall Health\nSystem,N,1.0,43.0,0.04,0.02,216.0,192.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238%,NaN,NaN,NaN,NaN,NaN
2,10006,Eliza Coffee Memorial Hospital,205 Marengo Street,Florence,AL,35631,Lifepoint Health,N,3.0,118.0,0.09,0.04,194.0,149.93,42.0,0.52,0.21,251.0,18265.0,0.61,0.25,137.0,297%,205%,241.0,0.51,0.18,0.09,0.07
3,10011,St. Vincents East,50 Medical Park East\nDrive,Birmingham,AL,35235,Ascension Health,N,2.0,30.0,0.06,0.04,168.0,130.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148%,NaN,NaN,NaN,NaN,NaN
4,10012,Dekalb Regional Medical\nCenter,200 Med Center Drive,Fort Payne,AL,35968,Quorum Health,N,2.0,18.0,0.03,0.02,114.0,94.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108%,NaN,NaN,NaN,NaN,NaN


(117301, 1325)


prvdr_num  rpt_rec_num_1  rpt_rec_num_2  rpt_rec_num_3 FY_BGN_DT_1  \
0      10001          58001        58002.0            NaN   01OCT1995   
1      10001          58002        58003.0            NaN   01OCT1996   
2      10001          58003        58004.0            NaN   01OCT1997   
3      10001          58004        79001.0            NaN   01OCT1998   
4      10001          79001        93411.0            NaN   01OCT1999   

  FY_BGN_DT_2 FY_BGN_DT_3 FY_END_DT_1 FY_END_DT_2 FY_END_DT_3  RPT_STUS_CD_1  \
0   01OCT1996         NaN   30SEP1996   30SEP1997         NaN              4   
1   01OCT1997         NaN   30SEP1997   30SEP1998         NaN              4   
2   01OCT1998         NaN   30SEP1998   30SEP1999         NaN              4   
3   01OCT1999         NaN   30SEP1999   30SEP2000         NaN              2   
4   01OCT2000         NaN   30SEP2000   30SEP2001         NaN              2   

   RPT_STUS_CD_2  RPT_STUS_CD_3       source_1       source_2 source_3  \
0            4.0            NaN  2552-96, 1996  2552-96, 1997      NaN   
1            4.0            NaN  2552-96, 1997  2552-96, 1998      NaN   
2            2.0            NaN  2552-96, 1998  2552-96, 1999      NaN   
3            2.0            NaN  2552-96, 1999  2552-96, 2000      NaN   
4            4.0            NaN  2552-96, 2000  2552-96, 2001      NaN   

    wt_cy_1   wt_cy_2  wt_cy_3  wt_rep_cy_1  wt_rep_cy_2  wt_rep_cy_3  \
0  0.748634  0.251366      NaN     0.748634     0.252055          NaN   
1  0.747945  0.252055      NaN     0.747945     0.252055          NaN   
2  0.747945  0.252055      NaN     0.747945     0.252055          NaN   
3  0.747945  0.252055      NaN     0.747945     0.251366          NaN   
4  0.748634  0.251366      NaN     0.748634     0.252055          NaN   

   wt_rep_cy_4    cy  expns_inp_subp_ipf_salary_only10  \
0          NaN  1996                               NaN   
1          NaN  1997                               NaN   
2          NaN  1998                               NaN   
3          NaN  1999                               NaN   
4          NaN  2000                               NaN   

   expns_inp_subp_irf_salary_only10  expns_anc_CT_salary_only10  \
0                               NaN                         NaN   
1                               NaN                         NaN   
2                               NaN                         NaN   
3                               NaN                         NaN   
4                               NaN                         NaN   

   expns_anc_MRI_salary_only10  expns_anc_cath_salary_only10  \
0                          NaN                           NaN   
1                          NaN                           NaN   
2                          NaN                           NaN   
3                          NaN                           NaN   
4                          NaN                           NaN   

   expns_anc_dvc_chgd_salary_only10  expns_outp_rhc_salary_only10  \
0                               NaN                           NaN   
1                               NaN                           NaN   
2                               NaN                           NaN   
3                               NaN                           NaN   
4                               NaN                           NaN   

   expns_outp_fqhc_salary_only10  expns_other_reim_salary_only10  \
0                            NaN                             NaN   
1                            NaN                             NaN   
2                            NaN                             NaN   
3                            NaN                             NaN   
4                            NaN                             NaN   

   expns_spec_pancr_salary_only10  expns_spec_intes_salary_only10  \
0                             NaN                             NaN   
1                             NaN                             NaN   
2                             N

#### Data Exploration

In [86]:
#Generated data profiling html reports but is very time consuming
#profile_pf_excel = ProfileReport(PF_excel, minimal=True)
#profile_rand_hcris = ProfileReport(rand_hcris, minimal=True)
#profile_pf_excel.to_file("pf_hospitals_output.html")
#profile_rand_hcris.to_file("rand_hcris_output.html")

In [87]:
PF_desc = PF_excel.describe()
display(PF_excel.describe())
rand_desc = rand_hcris.describe()
display(rand_desc)

,Medicare_provider_number,Zip_code,"Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)",Number_of_outpatient_services,Total_private_allowed_amount_for_outpatient_services_($_millions),Simulated_Medicare_allowed_amount_for_outpatient_services_($_millions),Relative_price_for_outpatient_services,Standardized_price_per_outpatient_service,Number_of_inpatient_stays,Total_private_allowed_amount_for_inpatient_services_($_millions),Simulated_Medicare_allowed_amount_for_inpatient_services_($_millions),Relative_price_for_inpatient_services,Standardized_price_per_inpatient_stay,Total_private_allowed_amount_for_inpatient_and_outpatient_services_($_millions),Simulated_Medicare_allowed_amount_for_inpatient_and_outpatient_services_($_millions),Relative_price_for_professional_inpatient_and_outpatient_services,Relative_price_for_inpatient_and_outpatient_services,Total_private_allowed_amount_for_facility_inpatient_and_outpatient_services_($_millions),Simulated_Medicare_allowed_amount_for_facility_inpatient_and_outpatient_services_($_millions),Total_private_allowed_amount_for_professional_inpatient_and_outpatient_services_($_millions),Simulated_Medicare_allowed_amount_for_professional_inpatient_and_outpatient_services_($_millions)
count,3112.000000,3112.000000,2853.000000,3078.000000,3078.000000,3078.000000,3078.000000,3078.000000,1620.00000,1620.000000,1620.000000,1620.000000,1620.000000,1586.000000,1586.000000,1586.000000,1586.000000,1586.000000,1586.000000,1586.000000,1586.000000
mean,265027.717224,52025.459190,3.060287,1779.175763,4.344886,1.656157,269.673814,287.673337,339.47284,8.397704,3.957969,230.018519,20867.461111,16.765328,7.153960,162.407945,249.354981,14.771274,5.966765,1.994124,1.186986
std,159293.540389,28190.692335,1.086644,6932.941664,16.928695,6.512202,109.916295,130.012982,1204.07905,29.365045,16.027683,186.157092,8813.133166,50.638278,23.966798,59.255204,76.202880,44.301604,20.633490,6.687558,3.488028
min,10001.000000,1040.000000,1.000000,11.000000,0.000000,0.000000,49.000000,56.940000,11.00000,0.050000,0.050000,44.000000,3115.000000,0.110000,0.090000,58.000000,48.000000,0.090000,0.070000,0.010000,0.010000
25%,131318.000000,30220.750000,2.000000,43.000000,0.090000,0.040000,191.000000,202.500000,21.00000,0.420000,0.210000,172.000000,15091.750000,1.030000,0.440000,126.000000,196.000000,0.910000,0.360000,0.120000,0.080000
50%,260024.000000,49090.500000,3.000000,139.000000,0.330000,0.130000,254.000000,266.480000,51.00000,1.190000,0.530000,215.000000,19244.500000,2.625000,1.090000,149.500000,242.000000,2.310000,0.900000,0.285000,0.190000
75%,390173.250000,76862.500000,4.000000,608.000000,1.440000,0.567500,327.000000,341.590000,176.50000,4.100000,1.790000,272.000000,24820.750000,9.450000,4.050000,177.750000,294.000000,8.435000,3.295000,1.017500,0.670000
max,670110.000000,99801.000000,5.000000,155155.000000,292.080000,129.850000,1176.000000,1668.530000,19906.00000,493.430000,295.800000,6962.000000,126518.000000,759.190000,369.410000,679.000000,815.000000,670.510000,322.040000,125.650000,47.370000


prvdr_num  rpt_rec_num_1  rpt_rec_num_2  rpt_rec_num_3  \
count  117301.000000  117301.000000   68305.000000     687.000000   
mean   263399.639517  336395.092915  358108.257287  307963.960699   
std    152893.263889  229791.323880  227705.943748  226480.877551   
min     10001.000000       7.000000      15.000000    1750.000000   
25%    140124.000000  146534.000000  163987.000000  132615.500000   
50%    250149.000000  283061.000000  292788.000000  230488.000000   
75%    390072.000000  594677.000000  611740.000000  565143.500000   
max    670129.000000  684242.000000  684241.000000  683916.000000   

       RPT_STUS_CD_1  RPT_STUS_CD_2  RPT_STUS_CD_3        wt_cy_1  \
count  117301.000000   68305.000000     687.000000  117301.000000   
mean        2.733446       2.709846       2.703057       0.712105   
std         1.104778       1.123914       1.147582       0.252782   
min         1.000000       1.000000       1.000000       0.002732   
25%         2.000000       2.000000       2.000000       0.495890   
50%         2.000000       2.000000       2.000000       0.747945   
75%         4.000000       4.000000       4.000000       1.000000   
max         5.000000       5.000000       5.000000       1.000000   

            wt_cy_2     wt_cy_3    wt_rep_cy_1   wt_rep_cy_2  wt_rep_cy_3  \
count  68305.000000  687.000000  117301.000000  68305.000000   687.000000   
mean       0.432910    0.297772       0.726141      0.454717     0.387920   
std        0.161455    0.148465       0.249315      0.187217     0.244710   
min        0.002740    0.002732       0.000000      0.002747     0.002732   
25%        0.252055    0.251366       0.495890      0.252055     0.252055   
50%        0.504109    0.252055       0.747945      0.504109     0.333333   
75%        0.504109    0.418033       1.000000      0.504109     0.504109   
max        0.997260    0.761643       1.000000      1.000000     1.000000   

       wt_rep_cy_4             cy  expns_inp_subp_ipf_salary_only10  \
count     6.000000  117301.000000                      1.077900e+04   
mean      0.258502    2007.321941                      2.825864e+06   
std       0.178373       6.904058                      3.680042e+06   
min       0.084931    1996.000000                     -1.341750e+03   
25%       0.145352    2001.000000                      9.401281e+05   
50%       0.209589    2007.000000                      1.726551e+06   
75%       0.313014    2013.000000                      3.346950e+06   
max       0.575471    2019.000000                      6.236944e+07   

       expns_inp_subp_irf_salary_only10  expns_anc_CT_salary_only10  \
count                      8.425000e+03                2.170800e+04   
mean                       1.963606e+06                6.348764e+05   
std                        2.066583e+06                7.154430e+05   
min                       -4.503200e+04               -2.242700e+04   
25%                        8.167819e+05                2.044842e+05   
50%                        1.394374e+06                4.661520e+05   
75%                        2.320240e+06                8.023150e+05   
max                        2.364427e+07                1.077605e+07   

       expns_anc_MRI_salary_only10  expns_anc_cath_salary_only10  \
count                 1.935300e+04                  1.389500e+04   
mean                  4.328796e+05                  1.619317e+06   
std                   6.957749e+05                  2.053238e+06   
min                  -1.713970e+04                 -1.167651e+05   
25%                   1.187953e+05                  5.437492e+05   
50%                   2.391958e+05                  1.107570e+06   
75%                   4.710040e+05                  1.969235e+06   
max                   1.330902e+07                  4.651501e+07   

       expns_anc_dvc_chgd_salary_only10  expns_outp_rhc_salary_only10  \
count                      1.800000e+02                  1.074700e+04   
mean                       1.13341

In [88]:
pf_nulls = pd.DataFrame(getPctMissing(PF_excel),columns=(["Null_Percentage"]))
pf_nulls.rename_axis("variable").reset_index()

rand_nulls = pd.DataFrame(getPctMissing(rand_hcris),columns=(["Null_Percentage"]))
rand_nulls.rename_axis("variable").reset_index()
display(pf_nulls.head())
display(rand_nulls.head())

,Null_Percentage
Medicare_provider_number,0.00
Hospital_name,0.03
Street_address,0.00
City,0.00
State,0.00


,Null_Percentage
prvdr_num,0.00
rpt_rec_num_1,0.00
rpt_rec_num_2,41.77
rpt_rec_num_3,99.41
FY_BGN_DT_1,0.00


#### Data Preparation

In [89]:
#IPPS
IPPS_prvdr_state_18 = pd.merge(IPPS_by_prvdr18, IPPS_by_state18, on=['STATE_DESC','DRG_DESC'], how="left")
IPPS_prvdr_state_18.rename(columns={"AVERAGE_COVERED_CHARGES": "STATE_AVERAGE_COVERED_CHARGES",
                   "AVERAGE_MEDICARE_PAYMENTS": "STATE_AVERAGE_MEDICARE_PAYMENTS",
                   "AVERAGE_MEDICARE_REIMBURSEMENT":"STATE_AVERAGE_MEDICARE_REIMBURSEMENT"},inplace=True)

IPPS_prvdr_state_nat_18 = pd.merge(IPPS_prvdr_state_18, IPPS_by_national18, on=['DRG_DESC'], how="left")
IPPS_prvdr_state_nat_18.rename(columns={"AVERAGE_COVERED_CHARGES": "NATIONAL_AVERAGE_COVERED_CHARGES",
                   "AVERAGE_MEDICARE_PAYMENTS": "NATIONAL_AVERAGE_MEDICARE_PAYMENTS",
                   "AVERAGE_MEDICARE_REIMBURSEMENT":"NATIONAL_AVERAGE_MEDICARE_REIMBURSEMENT"},inplace=True)

IPPS_prvdr_state_nat_18["COVERED_CHARGES_STATE_SCORE"] =   (IPPS_prvdr_state_nat_18["MEAN_COVERED_CHARGES"] - IPPS_prvdr_state_nat_18["STATE_AVERAGE_COVERED_CHARGES"])/IPPS_prvdr_state_nat_18["STATE_AVERAGE_COVERED_CHARGES"]
IPPS_prvdr_state_nat_18["MEDICARE_PAYMENTS_STATE_SCORE"] =   (IPPS_prvdr_state_nat_18["MEAN_MEDICARE_PAYMENTS"] - IPPS_prvdr_state_nat_18["STATE_AVERAGE_MEDICARE_PAYMENTS"])/IPPS_prvdr_state_nat_18["STATE_AVERAGE_MEDICARE_PAYMENTS"]  
IPPS_prvdr_state_nat_18["MEDICARE_REIMBURSEMENT_STATE_SCORE"] =   (IPPS_prvdr_state_nat_18["MEAN_MEDICARE_REIMBURSEMENT"] - IPPS_prvdr_state_nat_18["STATE_AVERAGE_MEDICARE_REIMBURSEMENT"])/IPPS_prvdr_state_nat_18["STATE_AVERAGE_MEDICARE_REIMBURSEMENT"]  

IPPS_prvdr_state_nat_18["COVERED_CHARGES_NATIONAL_SCORE"] =   (IPPS_prvdr_state_nat_18["MEAN_COVERED_CHARGES"] - IPPS_prvdr_state_nat_18["NATIONAL_AVERAGE_COVERED_CHARGES"])/IPPS_prvdr_state_nat_18["NATIONAL_AVERAGE_COVERED_CHARGES"]
IPPS_prvdr_state_nat_18["MEDICARE_PAYMENTS_NATIONAL_SCORE"] =   (IPPS_prvdr_state_nat_18["MEAN_MEDICARE_PAYMENTS"] - IPPS_prvdr_state_nat_18["NATIONAL_AVERAGE_MEDICARE_PAYMENTS"])/IPPS_prvdr_state_nat_18["NATIONAL_AVERAGE_MEDICARE_PAYMENTS"]  
IPPS_prvdr_state_nat_18["MEDICARE_REIMBURSEMENT_NATIONAL_SCORE"] =   (IPPS_prvdr_state_nat_18["MEAN_MEDICARE_REIMBURSEMENT"] - IPPS_prvdr_state_nat_18["NATIONAL_AVERAGE_MEDICARE_REIMBURSEMENT"])/IPPS_prvdr_state_nat_18["NATIONAL_AVERAGE_MEDICARE_REIMBURSEMENT"]

IPPS_prvdr_state_nat_18 = IPPS_prvdr_state_nat_18[["PROVIDER_ID","DRG_DESC","COVERED_CHARGES_STATE_SCORE","MEDICARE_PAYMENTS_STATE_SCORE","MEDICARE_REIMBURSEMENT_STATE_SCORE","COVERED_CHARGES_NATIONAL_SCORE","MEDICARE_PAYMENTS_NATIONAL_SCORE","MEDICARE_REIMBURSEMENT_NATIONAL_SCORE"]]

#FILTER IPPS DATA ON MOST COMMON DRG I.E "871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W MCC"
# AND DRG "291 - HEART FAILURE & SHOCK W MCC"
IPPS_prvdr_state_nat_18_871 = IPPS_prvdr_state_nat_18.loc[IPPS_prvdr_state_nat_18['DRG_DESC'] == "871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W MCC"]
IPPS_prvdr_state_nat_18_291 = IPPS_prvdr_state_nat_18.loc[IPPS_prvdr_state_nat_18['DRG_DESC'] == "291 - HEART FAILURE & SHOCK W MCC"]
 
IPPS_prvdr_state_nat_18_871_291 = pd.merge(IPPS_prvdr_state_nat_18_871, IPPS_prvdr_state_nat_18_291, on=['PROVIDER_ID'], how="outer", suffixes=('_DRG871', '_DRG291'))
IPPS_prvdr_state_nat_18_871_291 = drop_unwanted_columns(IPPS_prvdr_state_nat_18_871_291,['DRG_DESC_DRG871','DRG_DESC_DRG291'])
print(IPPS_prvdr_state_nat_18_871_291.shape)
IPPS_prvdr_state_nat_18_871_291.head()

(2874, 13)


,PROVIDER_ID,COVERED_CHARGES_STATE_SCORE_DRG871,MEDICARE_PAYMENTS_STATE_SCORE_DRG871,MEDICARE_REIMBURSEMENT_STATE_SCORE_DRG871,COVERED_CHARGES_NATIONAL_SCORE_DRG871,MEDICARE_PAYMENTS_NATIONAL_SCORE_DRG871,MEDICARE_REIMBURSEMENT_NATIONAL_SCORE_DRG871,COVERED_CHARGES_STATE_SCORE_DRG291,MEDICARE_PAYMENTS_STATE_SCORE_DRG291,MEDICARE_REIMBURSEMENT_STATE_SCORE_DRG291,COVERED_CHARGES_NATIONAL_SCORE_DRG291,MEDICARE_PAYMENTS_NATIONAL_SCORE_DRG291,MEDICARE_REIMBURSEMENT_NATIONAL_SCORE_DRG291
0,10001,-0.134898,-0.094744,-0.064342,-0.129920,-0.275816,-0.263235,0.010861,-0.054909,-0.020237,-0.107015,-0.246323,-0.229200
1,10005,-0.618456,-0.067300,-0.100913,-0.616260,-0.253862,-0.292032,-0.500387,-0.064118,-0.061856,-0.558647,-0.253667,-0.261943
2,10006,-0.277163,-0.176073,-0.168977,-0.273004,-0.340878,-0.345627,-0.162458,-0.150186,-0.139908,-0.260123,-0.322303,-0.323348
3,10007,-0.703065,-0.116568,-0.072164,-0.701356,-0.293275,-0.269394,-0.565856,-0.106978,-0.085010,-0.616481,-0.287847,-0.280159
4,10011,-0.078664,-0.011334,-0.011489,-0.073362,-0.209090,-0.221617,0.076202,0.081096,0.035767,-0.049294,-0.137864,-0.185141


In [90]:
# OPPS
OPPS_by_prvdr18.columns = column_format_newline(OPPS_by_prvdr18.columns)
OPPS_by_state18.columns = column_format_newline(OPPS_by_state18.columns)
OPPS_by_national18.columns = column_format_newline(OPPS_by_national18.columns)
OPPS_prvdr_state_18 = pd.merge(OPPS_by_prvdr18, OPPS_by_state18, on=['Provider_State','APC_Description'], how="left",suffixes=["_PRVDR","_STATE"])
OPPS_prvdr_state_nat_18 = pd.merge(OPPS_prvdr_state_18, OPPS_by_national18, on=['APC_Description'], how="left")
# Filter for most common APC Description i.e "Comprehensive Observation Services"
OPPS_prvdr_state_nat_18 = OPPS_prvdr_state_nat_18.loc[OPPS_prvdr_state_nat_18['APC_Description'] == "Comprehensive Observation Services"]
OPPS_prvdr_state_nat_18 = OPPS_prvdr_state_nat_18[["Provider_ID","Beneficiaries_PRVDR","Comprehensive_APC_Services_PRVDR","Average_Estimated_Total_Submitted_Charges_PRVDR","Average_Medicare_Allowed_Amount_PRVDR","Average_Medicare_Payment_Amount_PRVDR","Beneficiaries_STATE","Comprehensive_APC_Services_STATE","Average_Estimated_Total_Submitted_Charges_STATE","Average_Medicare_Allowed_Amount_STATE","Average_Medicare_Payment_Amount_STATE","Beneficiaries","Comprehensive_APC_Services","Average_Estimated_Total_Submitted_Charges","Average_Medicare_Allowed_Amount","Average_Medicare_Payment_Amount"]]
print(OPPS_prvdr_state_nat_18.shape)
display(OPPS_prvdr_state_nat_18.head())
OPPS_prvdr_state_nat_18["OPPS_Beneficiaries_Perc"] =   (OPPS_prvdr_state_nat_18["Beneficiaries_PRVDR"]*100.0)/OPPS_prvdr_state_nat_18["Beneficiaries"]
OPPS_prvdr_state_nat_18["OPPS_Comprehensive_APC_Services_Perc"] =   (OPPS_prvdr_state_nat_18["Comprehensive_APC_Services_PRVDR"]*100.0)/OPPS_prvdr_state_nat_18["Comprehensive_APC_Services"]
OPPS_prvdr_state_nat_18["OPPS_Total_Submitted_Charges_STATE_SCORE"] =   (OPPS_prvdr_state_nat_18["Average_Estimated_Total_Submitted_Charges_PRVDR"] - OPPS_prvdr_state_nat_18["Average_Estimated_Total_Submitted_Charges_STATE"])/OPPS_prvdr_state_nat_18["Average_Estimated_Total_Submitted_Charges_STATE"]  
OPPS_prvdr_state_nat_18["OPPS_Medicare_Allowed_Amount_STATE_SCORE"] =   (OPPS_prvdr_state_nat_18["Average_Medicare_Allowed_Amount_PRVDR"] - OPPS_prvdr_state_nat_18["Average_Medicare_Allowed_Amount_STATE"])/OPPS_prvdr_state_nat_18["Average_Medicare_Allowed_Amount_STATE"]  
OPPS_prvdr_state_nat_18["OPPS_Medicare_Payment_Amount_STATE_SCORE"] =   (OPPS_prvdr_state_nat_18["Average_Medicare_Payment_Amount_PRVDR"] - OPPS_prvdr_state_nat_18["Average_Medicare_Payment_Amount_STATE"])/OPPS_prvdr_state_nat_18["Average_Medicare_Payment_Amount_STATE"]  
OPPS_prvdr_state_nat_18["OPPS_Total_Submitted_Charges_NATIONAL_SCORE"] =   (OPPS_prvdr_state_nat_18["Average_Estimated_Total_Submitted_Charges_PRVDR"] - OPPS_prvdr_state_nat_18["Average_Estimated_Total_Submitted_Charges"])/OPPS_prvdr_state_nat_18["Average_Estimated_Total_Submitted_Charges"]  
OPPS_prvdr_state_nat_18["OPPS_Medicare_Allowed_Amount_NATIONAL_SCORE"] =   (OPPS_prvdr_state_nat_18["Average_Medicare_Allowed_Amount_PRVDR"] - OPPS_prvdr_state_nat_18["Average_Medicare_Allowed_Amount"])/OPPS_prvdr_state_nat_18["Average_Medicare_Allowed_Amount"]  
OPPS_prvdr_state_nat_18["OPPS_Medicare_Payment_Amount_NATIONAL_SCORE"] =   (OPPS_prvdr_state_nat_18["Average_Medicare_Payment_Amount_PRVDR"] - OPPS_prvdr_state_nat_18["Average_Medicare_Payment_Amount"])/OPPS_prvdr_state_nat_18["Average_Medicare_Payment_Amount"]  

OPPS_prvdr_state_nat_18 = OPPS_prvdr_state_nat_18[["Provider_ID","OPPS_Beneficiaries_Perc","OPPS_Comprehensive_APC_Services_Perc","OPPS_Total_Submitted_Charges_STATE_SCORE","OPPS_Medicare_Allowed_Amount_STATE_SCORE","OPPS_Medicare_Payment_Amount_STATE_SCORE","OPPS_Total_Submitted_Charges_NATIONAL_SCORE","OPPS_Medicare_Allowed_Amount_NATIONAL_SCORE","OPPS_Medicare_Payment_Amount_NATIONAL_SCORE"]]
print(OPPS_prvdr_state_nat_18.shape)
display(OPPS_prvdr_state_nat_18.head())


(2932, 16)


,Provider_ID,Beneficiaries_PRVDR,Comprehensive_APC_Services_PRVDR,Average_Estimated_Total_Submitted_Charges_PRVDR,Average_Medicare_Allowed_Amount_PRVDR,Average_Medicare_Payment_Amount_PRVDR,Beneficiaries_STATE,Comprehensive_APC_Services_STATE,Average_Estimated_Total_Submitted_Charges_STATE,Average_Medicare_Allowed_Amount_STATE,Average_Medicare_Payment_Amount_STATE,Beneficiaries,Comprehensive_APC_Services,Average_Estimated_Total_Submitted_Charges,Average_Medicare_Allowed_Amount,Average_Medicare_Payment_Amount
39,10001,705.0,755,15370.995377,1992.030636,1584.815868,18301.0,19856.0,13869.31094,1937.919138,1534.361167,1283415,1429341,16727.059293,2293.63937,1821.28938
71,10005,206.0,216,7734.270972,1973.092639,1562.370556,18301.0,19856.0,13869.31094,1937.919138,1534.361167,1283415,1429341,16727.059293,2293.63937,1821.28938
101,10006,431.0,453,9729.452627,1856.536843,1474.405982,18301.0,19856.0,13869.31094,1937.919138,1534.361167,1283415,1429341,16727.059293,2293.63937,1821.28938
109,10007,167.0,178,6126.810169,1858.145281,1477.055281,18301.0,19856.0,13869.31094,1937.919138,1534.361167,1283415,1429341,16727.059293,2293.63937,1821.28938
110,10008,97.0,118,11313.856610,1866.475000,1480.406356,18301.0,19856.0,13869.31094,1937.919138,1534.361167,1283415,1429341,16727.059293,2293.63937,1821.28938


(2932, 9)


,Provider_ID,OPPS_Beneficiaries_Perc,OPPS_Comprehensive_APC_Services_Perc,OPPS_Total_Submitted_Charges_STATE_SCORE,OPPS_Medicare_Allowed_Amount_STATE_SCORE,OPPS_Medicare_Payment_Amount_STATE_SCORE,OPPS_Total_Submitted_Charges_NATIONAL_SCORE,OPPS_Medicare_Allowed_Amount_NATIONAL_SCORE,OPPS_Medicare_Payment_Amount_NATIONAL_SCORE
39,10001,0.054932,0.052822,0.108274,0.027922,0.032883,-0.081070,-0.131498,-0.129839
71,10005,0.016051,0.015112,-0.442346,0.018150,0.018255,-0.537619,-0.139755,-0.142162
101,10006,0.033582,0.031693,-0.298491,-0.041995,-0.039075,-0.418341,-0.190572,-0.190460
109,10007,0.013012,0.012453,-0.558247,-0.041165,-0.037348,-0.633719,-0.189870,-0.189006
110,10008,0.007558,0.008256,-0.184252,-0.036866,-0.035164,-0.323620,-0.186239,-0.187166


In [91]:
# Filter randhcris on 2018 & 2019 for training & Scoring
rand_2018 = rand_hcris.loc[rand_hcris['cy'] == int(Training_rand)]
rand_2019 = rand_hcris.loc[rand_hcris['cy'] == int(Scoring_rand)]
print("rand_2018",rand_2018.shape)
print("rand_2019",rand_2019.shape)

rand_2018 (4694, 1325)
rand_2019 (4639, 1325)


In [92]:
# Comparing null percentage between 2018 and 2019
rand_2018_nulls = pd.DataFrame(getPctMissing(rand_2018),columns=(["Null%_2018"]))
rand_2018_nulls.rename_axis("variable").reset_index()
rand_2019_nulls = pd.DataFrame(getPctMissing(rand_2019),columns=(["Null%_2019"]))
rand_2019_nulls.rename_axis("variable").reset_index()
rand_2018_nulls.join(rand_2019_nulls).head()
# Almost similar in both years
# We will use 2018 in model training/validation and 2019 will be used for scoring

,Null%_2018,Null%_2019
FY_BGN_DT_1,0.00,0.00
FY_BGN_DT_2,39.31,69.76
FY_BGN_DT_3,99.32,99.85
FY_END_DT_1,0.00,0.00
FY_END_DT_2,39.31,69.76


In [93]:
hospital_data_2018 = pd.merge(PF_excel, rand_2018, left_on='Medicare_provider_number', right_on='prvdr_num')
hospital_data_2018 =  pd.merge(hospital_data_2018, IPPS_prvdr_state_nat_18_871_291, left_on='Medicare_provider_number', right_on='PROVIDER_ID',how="left")
hospital_data_2018 = drop_unwanted_columns(hospital_data_2018,['PROVIDER_ID'])
hospital_data_2018 = pd.merge(hospital_data_2018, OPPS_prvdr_state_nat_18, left_on='Medicare_provider_number', right_on='Provider_ID',how="left")
hospital_data_2018 = drop_unwanted_columns(hospital_data_2018,['Provider_ID'])

hospital_data_2019 = pd.merge(PF_excel, rand_2019, left_on='Medicare_provider_number', right_on='prvdr_num')
hospital_data_2019 =  pd.merge(hospital_data_2019, IPPS_prvdr_state_nat_18_871_291, left_on='Medicare_provider_number', right_on='PROVIDER_ID',how="left")
hospital_data_2019 = drop_unwanted_columns(hospital_data_2019,['PROVIDER_ID'])
hospital_data_2019 = pd.merge(hospital_data_2019, OPPS_prvdr_state_nat_18, left_on='Medicare_provider_number', right_on='Provider_ID',how="left")
hospital_data_2019 = drop_unwanted_columns(hospital_data_2019,['Provider_ID'])


print("2018",hospital_data_2018.shape)
print("2019",hospital_data_2019.shape)
display(hospital_data_2018.head())
display(hospital_data_2019.head())

2018 (3084, 1374)
2019 (3038, 1374)


Medicare_provider_number                      Hospital_name  \
0                     10001  Southeast Alabama Medical\nCenter   
1                     10005   Marshall Medical Center -\nSouth   
2                     10006     Eliza Coffee Memorial Hospital   
3                     10011                  St. Vincents East   
4                     10012    Dekalb Regional Medical\nCenter   

                Street_address        City State  Zip_code  \
0       1108 Ross Clark Circle      Dothan    AL     36301   
1  2505 U S Highway 431\nNorth        Boaz    AL     35957   
2           205 Marengo Street    Florence    AL     35631   
3  50 Medical Park East\nDrive  Birmingham    AL     35235   
4         200 Med Center Drive  Fort Payne    AL     35968   

  Hospital_system_or,_if_independent,_IPPS/CAH  \
0                             Southeast Health   
1                      Marshall Health\nSystem   
2                             Lifepoint Health   
3                             Ascension Health   
4                                Quorum Health   

  Is_hospital_a_critical_access_hospital_(Y/N)?  \
0                                             N   
1                                             N   
2                                             N   
3                                             N   
4                                             N   

   Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)  \
0                                                2.0                 
1                                                1.0                 
2                                                3.0                 
3                                                2.0                 
4                                                2.0                 

   Number_of_outpatient_services  \
0                           54.0   
1                           43.0   
2                          118.0   
3                           30.0   
4                           18.0   

   Total_private_allowed_amount_for_outpatient_services_($_millions)  \
0                                               0.11                   
1                                               0.04                   
2                                               0.09                   
3                                               0.06                   
4                                               0.03                   

   Simulated_Medicare_allowed_amount_for_outpatient_services_($_millions)  \
0                                               0.09                        
1                                               0.02                        
2                                               0.04                        
3                                               0.04                        
4                                               0.02                        

   Relative_price_for_outpatient_services  \
0                                   122.0   
1                                   216.0   
2                                   194.0   
3                                   168.0   
4                                   114.0   

   Standardized_price_per_outpatient_service  Number_of_inpatient_stays  \
0                                      92.34                        NaN   
1                                     192.11                        NaN   
2                                     149.93                       42.0   
3                                     130.33                        NaN   
4                                      94.36                        NaN   

   Total_private_allowed_amount_for_inpatient_services_($_millions)  \
0                                                NaN                  
1                                                NaN                  
2                                               0.52                  
3                                                NaN                  
4             

Medicare_provider_number                      Hospital_name  \
0                     10001  Southeast Alabama Medical\nCenter   
1                     10005   Marshall Medical Center -\nSouth   
2                     10006     Eliza Coffee Memorial Hospital   
3                     10011                  St. Vincents East   
4                     10012    Dekalb Regional Medical\nCenter   

                Street_address        City State  Zip_code  \
0       1108 Ross Clark Circle      Dothan    AL     36301   
1  2505 U S Highway 431\nNorth        Boaz    AL     35957   
2           205 Marengo Street    Florence    AL     35631   
3  50 Medical Park East\nDrive  Birmingham    AL     35235   
4         200 Med Center Drive  Fort Payne    AL     35968   

  Hospital_system_or,_if_independent,_IPPS/CAH  \
0                             Southeast Health   
1                      Marshall Health\nSystem   
2                             Lifepoint Health   
3                             Ascension Health   
4                                Quorum Health   

  Is_hospital_a_critical_access_hospital_(Y/N)?  \
0                                             N   
1                                             N   
2                                             N   
3                                             N   
4                                             N   

   Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)  \
0                                                2.0                 
1                                                1.0                 
2                                                3.0                 
3                                                2.0                 
4                                                2.0                 

   Number_of_outpatient_services  \
0                           54.0   
1                           43.0   
2                          118.0   
3                           30.0   
4                           18.0   

   Total_private_allowed_amount_for_outpatient_services_($_millions)  \
0                                               0.11                   
1                                               0.04                   
2                                               0.09                   
3                                               0.06                   
4                                               0.03                   

   Simulated_Medicare_allowed_amount_for_outpatient_services_($_millions)  \
0                                               0.09                        
1                                               0.02                        
2                                               0.04                        
3                                               0.04                        
4                                               0.02                        

   Relative_price_for_outpatient_services  \
0                                   122.0   
1                                   216.0   
2                                   194.0   
3                                   168.0   
4                                   114.0   

   Standardized_price_per_outpatient_service  Number_of_inpatient_stays  \
0                                      92.34                        NaN   
1                                     192.11                        NaN   
2                                     149.93                       42.0   
3                                     130.33                        NaN   
4                                      94.36                        NaN   

   Total_private_allowed_amount_for_inpatient_services_($_millions)  \
0                                                NaN                  
1                                                NaN                  
2                                               0.52                  
3                                                NaN                  
4             

In [94]:
hospital_data_2018_null = pd.DataFrame(getPctMissing(hospital_data_2018),columns=(["Null%"])).rename_axis("variable").reset_index()
hospital_data_2018_null
hospital_red1_df = hospital_data_2018_null.loc[hospital_data_2018_null['Null%'] < 10].reset_index()
# Reduced variables to 316 by removing columns with > 10% nulls
print(hospital_red1_df.shape)
hospital_red1_df.head()

(316, 3)


,index,variable,Null%
0,0,Medicare_provider_number,0.00
1,1,Hospital_name,0.03
2,2,Street_address,0.00
3,3,City,0.00
4,4,State,0.00


In [95]:
#Drop unwanted variables and create dummy variables
#Drop target NaN rows
reducednull_variableset = list(hospital_red1_df["variable"])
#Following variables had around 20% nulls before null replacement but including them as they might have relationship with targt
#Created these variables from IPPS/OPPS
reducednull_variableset.extend(["COVERED_CHARGES_STATE_SCORE_DRG871","MEDICARE_PAYMENTS_STATE_SCORE_DRG871","MEDICARE_REIMBURSEMENT_STATE_SCORE_DRG871","COVERED_CHARGES_NATIONAL_SCORE_DRG871","MEDICARE_PAYMENTS_NATIONAL_SCORE_DRG871","MEDICARE_REIMBURSEMENT_NATIONAL_SCORE_DRG871","COVERED_CHARGES_STATE_SCORE_DRG291","MEDICARE_PAYMENTS_STATE_SCORE_DRG291","MEDICARE_REIMBURSEMENT_STATE_SCORE_DRG291","COVERED_CHARGES_NATIONAL_SCORE_DRG291","MEDICARE_PAYMENTS_NATIONAL_SCORE_DRG291","MEDICARE_REIMBURSEMENT_NATIONAL_SCORE_DRG291","OPPS_Beneficiaries_Perc","OPPS_Comprehensive_APC_Services_Perc","OPPS_Total_Submitted_Charges_STATE_SCORE","OPPS_Medicare_Allowed_Amount_STATE_SCORE","OPPS_Medicare_Payment_Amount_STATE_SCORE","OPPS_Total_Submitted_Charges_NATIONAL_SCORE","OPPS_Medicare_Allowed_Amount_NATIONAL_SCORE","OPPS_Medicare_Payment_Amount_NATIONAL_SCORE"])
redN_hospital_data_2018 = hospital_data_2018[reducednull_variableset]
redN_hospital_data_2019 = hospital_data_2019[reducednull_variableset]

redN_hospital_data_2018 = pd.get_dummies(redN_hospital_data_2018,columns=[
"any_home_office_costs"
,"critical_access_hosp_hcr"
,"teach_hosp_hcr"
,"receives_dsh_only10"
,"teach_hosp_nurs_alld_hcr_only10"
#,"cbsa_cnty_status"
#,"teach_hosp_addl_FTEs_hcr_only10"
,"receives_mdcd_DSH_YN_only10"
,"ipps_hosp_pos"
,"rural_urban"],drop_first=True) 

redN_hospital_data_2019 = pd.get_dummies(redN_hospital_data_2019,columns=[
"any_home_office_costs"
,"critical_access_hosp_hcr"
,"teach_hosp_hcr"
,"receives_dsh_only10"
,"teach_hosp_nurs_alld_hcr_only10"
#,"cbsa_cnty_status"
#,"teach_hosp_addl_FTEs_hcr_only10"
,"receives_mdcd_DSH_YN_only10"
,"ipps_hosp_pos"
,"rural_urban"],drop_first=True) 

dropcol_list = [
'Street_address'
,'City'
,'Zip_code'
,'Hospital_system_or,_if_independent,_IPPS/CAH'
,'Total_private_allowed_amount_for_outpatient_services_($_millions)'
,'Simulated_Medicare_allowed_amount_for_outpatient_services_($_millions)'
,'Standardized_price_per_outpatient_service'
,'Relative_price_for_outpatient_facility_services'
,'rpt_rec_num_1'
,'FY_BGN_DT_1'
,'FY_END_DT_1'
,'RPT_STUS_CD_1'
,'source_1'
,'wt_cy_1'
,'wt_rep_cy_1'
,'cy'
,'hosp_city_hcr'
,'hosp_name_hcr'
,'hosp_street_address_hcr'
,'hosp_zip5_hcr'
,'fips_st_cnty'
,'hosp_name_pos'
,'hosp_city_pos'
,'hosp_street_address_pos'
,'hosp_zip5_pos'
,'PRVDR_CTGRY_CD'
,'PRVDR_CTGRY_CD_formatted'
,'PRVDR_CTGRY_SBTYP_CD'
,'PRVDR_CTGRY_SBTYP_CD_formatted'
,'state_name'
,'state_abbreviation'
,'county_name'
,'cbsa'
,'cbsa_name'
,'rural_urban_desc'
,'n_hosp_reporting'
,'prvdr_num'
,"Is_hospital_a_critical_access_hospital_(Y/N)?"
,"critical_access_hosp_pos"
]
redN_hospital_data_2018 = drop_unwanted_columns(redN_hospital_data_2018,dropcol_list)
redN_hospital_data_2019 = drop_unwanted_columns(redN_hospital_data_2019,dropcol_list)
redN_hospital_data_2018 = redN_hospital_data_2018.loc[pd.notnull(redN_hospital_data_2018.Relative_price_for_outpatient_services)]


In [96]:
redN_hospital_data_2018.loc[redN_hospital_data_2018['Medicare_provider_number'] == 670073]

,Medicare_provider_number,Hospital_name,State,"Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)",Number_of_outpatient_services,Relative_price_for_outpatient_services,tot_expenses_employee_benefits,tot_expenses_admin_and_general,tot_expenses_housekeeping,tot_expenses_dietary,capasset_bldngsfixtrs_end10only,capasset_moveequip_end10only,capasset_subtotal_end10only,capasset_total_end10only,net_expenses_anc_dvc_chgd_only10,totcosts_devices_chgd,inpat_charges_devices_chgd,inpat_charges_total,outpat_charges_devices_chgd,outpat_charges_total,mdcd_net_revenue_only10,chgs_uninsured_charity_only10,costs_uninsured_charity_only10,uncomp_uninsured_charity_only10,baddebt_hospcmplx_only10,mdcr_baddebt_hospcmplx_only10,mdcr_baddebt_ln2701_only10,nonmdcr_baddebt_hospcmplx_only10,nonmdcr_baddebt_costs_only10,nonmdcr_uncomp_costs_only10,unreimb_uncomp_costs_only10,chgs_charity_patients_only10,costs_charity_patients_only10,uncomp_charity_patients_only10,mdcr_inpat_discharges_HMO_only10,expns_admin_and_general_salary,expns_nursing_admin_salary,expns_inp_adultspeds_salary,expns_anc_oper_room_salary,expns_anc_radiology_diag_salary,expns_anc_lab_salary,expns_anc_resp_therapy_salary,expns_outp_emergency_salary,expns_subtotal_salary,total_salaries,expenses_empeebens_other,expns_admin_and_general_other,expns_housekeeping_other,expns_dietary_other,total_other_expenses,capasset_bldngsfixtrs_beginning,capasset_moveequip_beginning,capasset_subtotal_beginning,capasset_total_beginning,capasset_moveequip_purchases,capasset_subtotal_purchases,capasset_total_purchases,net_expenses_cap_bldgs_fixtures,net_expenses_cap_movable_equip,net_expenses_employee_benefits,net_expenses_admin_and_general,net_expenses_operation_of_plant,net_expenses_laundry_and_linen,net_expenses_housekeeping,net_expenses_dietary,net_expenses_nursing_admin,net_expenses_medical_records,net_expenses_inp_adultspeds,net_expenses_anc_oper_room,net_expenses_anc_radiology_diag,net_expenses_anc_lab,net_expenses_anc_resp_therapy,net_expenses_anc_phys_therapy,net_expenses_anc_medsupps_chgd,net_expenses_anc_drgs_chgd,net_expenses_outp_emergency,net_expenses_subtotal,net_expenses_total,totcosts_adultspeds,totcosts_oper_room,totcosts_radiology_diag,totcosts_lab,totcosts_resp_therapy,totcosts_phys_therapy,totcosts_medsupps_chgd,totcosts_drgs_chgd,totcosts_emergency,totcosts_subtotal,totcosts_total,inpat_charges_adultspeds,inpat_charges_oper_room,inpat_charges_radiology_diag,inpat_charges_lab,inpat_charges_resp_therapy,inpat_charges_phys_therapy,inpat_charges_medsupps_chgd,inpat_charges_drgs_chgd,inpat_charges_emergency,inpat_charges_subtotal,outpat_charges_oper_room,outpat_charges_radiology_diag,outpat_charges_lab,outpat_charges_resp_therapy,outpat_charges_phys_therapy,outpat_charges_medsupps_chgd,outpat_charges_drgs_chgd,outpat_charges_emergency,outpat_charges_obs_beds,outpat_charges_subtotal,mdcr_outpat_charges,mdcr_outpat_costs,mdcr_inpat_costs,mdcr_inpat_charges_adpeds,mdcr_inpat_charges_anc_outp_oth,mdcr_outpat_lesser_costchg,gross_patient_rev_inpat_hosp,gross_patient_rev_inpat_anc,gross_patient_rev_inpat,gross_patient_rev_outpat,gross_patient_rev,allowances_discounts,net_patient_rev,operating_expenses,net_income_srvcs_patients,other_income_other,other_income_all,net_income_srvcs_patients_other,net_income,mdcd_charges,mdcd_costs,bed_days_available,mdcr_hmo_inpat_days,mdcr_inpat_days,mdcd_inpat_days,all_inpat_days,mdcr_inpat_discharges_ln1,mdcr_inpat_discharges,mdcd_inpat_discharges_ln1,mdcd_inpat_discharges,all_inpat_discharges_ln1,all_inpat_discharges,wt_cy,n_hosp_years_reporting,source_2552_10,cash_on_hand_and_in_banks,accounts_receivable,inventory,tot_cur_assets_genfund,tot_fixed_assets_genfund,tot_other_assets_genfund,tot_assets_genfund,tot_cur_liab_genfund,tot_longterm_liab_genfund,tot_liab_genfund,tot_fundbal_genfund,tot_liab_fundbal_genfund,cost_to_charge_ratio_for_uncomp,adult_peds_beds,tot_adult_peds_beds,beds,beds_incl_subprov,employees_FTEs,type_hosp,ownership_nonprofit,ownersh

In [97]:
# Null Replacement
# Quality star rating replace by State mode
redN_hospital_data_2018['Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)'] = redN_hospital_data_2018['Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)'].fillna(redN_hospital_data_2018.groupby('State')['Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)'].transform(lambda x: x.value_counts().idxmax()))
redN_hospital_data_2019['Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)'] = redN_hospital_data_2019['Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)'].fillna(redN_hospital_data_2019.groupby('State')['Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)'].transform(lambda x: x.value_counts().idxmax()))


# Numeric variables replaced by State averages
for col in redN_hospital_data_2018.columns:
    if(col not in ("Medicare_provider_number","Hospital_name","State","Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)","Number_of_outpatient_services","Relative_price_for_outpatient_services")):
        #for columns "expns_nursing_admin_salary" and "net_expenses_nursing_admin" we replace by national average for NY state
        #since all state values are NaN
        redN_hospital_data_2018[col] = redN_hospital_data_2018[col].fillna(redN_hospital_data_2018.groupby('State')[col].transform('mean'))
        redN_hospital_data_2019[col] = redN_hospital_data_2019[col].fillna(redN_hospital_data_2019.groupby('State')[col].transform('mean'))
        
redN_hospital_data_2018['expns_nursing_admin_salary'].fillna((redN_hospital_data_2018['expns_nursing_admin_salary'].mean()), inplace=True)        
redN_hospital_data_2018['net_expenses_nursing_admin'].fillna((redN_hospital_data_2018['net_expenses_nursing_admin'].mean()), inplace=True)
redN_hospital_data_2019['expns_nursing_admin_salary'].fillna((redN_hospital_data_2019['expns_nursing_admin_salary'].mean()), inplace=True)        
redN_hospital_data_2019['net_expenses_nursing_admin'].fillna((redN_hospital_data_2019['net_expenses_nursing_admin'].mean()), inplace=True)

In [98]:
redN_hospital_data_2018.loc[redN_hospital_data_2018['Medicare_provider_number'] == 670073]

,Medicare_provider_number,Hospital_name,State,"Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)",Number_of_outpatient_services,Relative_price_for_outpatient_services,tot_expenses_employee_benefits,tot_expenses_admin_and_general,tot_expenses_housekeeping,tot_expenses_dietary,capasset_bldngsfixtrs_end10only,capasset_moveequip_end10only,capasset_subtotal_end10only,capasset_total_end10only,net_expenses_anc_dvc_chgd_only10,totcosts_devices_chgd,inpat_charges_devices_chgd,inpat_charges_total,outpat_charges_devices_chgd,outpat_charges_total,mdcd_net_revenue_only10,chgs_uninsured_charity_only10,costs_uninsured_charity_only10,uncomp_uninsured_charity_only10,baddebt_hospcmplx_only10,mdcr_baddebt_hospcmplx_only10,mdcr_baddebt_ln2701_only10,nonmdcr_baddebt_hospcmplx_only10,nonmdcr_baddebt_costs_only10,nonmdcr_uncomp_costs_only10,unreimb_uncomp_costs_only10,chgs_charity_patients_only10,costs_charity_patients_only10,uncomp_charity_patients_only10,mdcr_inpat_discharges_HMO_only10,expns_admin_and_general_salary,expns_nursing_admin_salary,expns_inp_adultspeds_salary,expns_anc_oper_room_salary,expns_anc_radiology_diag_salary,expns_anc_lab_salary,expns_anc_resp_therapy_salary,expns_outp_emergency_salary,expns_subtotal_salary,total_salaries,expenses_empeebens_other,expns_admin_and_general_other,expns_housekeeping_other,expns_dietary_other,total_other_expenses,capasset_bldngsfixtrs_beginning,capasset_moveequip_beginning,capasset_subtotal_beginning,capasset_total_beginning,capasset_moveequip_purchases,capasset_subtotal_purchases,capasset_total_purchases,net_expenses_cap_bldgs_fixtures,net_expenses_cap_movable_equip,net_expenses_employee_benefits,net_expenses_admin_and_general,net_expenses_operation_of_plant,net_expenses_laundry_and_linen,net_expenses_housekeeping,net_expenses_dietary,net_expenses_nursing_admin,net_expenses_medical_records,net_expenses_inp_adultspeds,net_expenses_anc_oper_room,net_expenses_anc_radiology_diag,net_expenses_anc_lab,net_expenses_anc_resp_therapy,net_expenses_anc_phys_therapy,net_expenses_anc_medsupps_chgd,net_expenses_anc_drgs_chgd,net_expenses_outp_emergency,net_expenses_subtotal,net_expenses_total,totcosts_adultspeds,totcosts_oper_room,totcosts_radiology_diag,totcosts_lab,totcosts_resp_therapy,totcosts_phys_therapy,totcosts_medsupps_chgd,totcosts_drgs_chgd,totcosts_emergency,totcosts_subtotal,totcosts_total,inpat_charges_adultspeds,inpat_charges_oper_room,inpat_charges_radiology_diag,inpat_charges_lab,inpat_charges_resp_therapy,inpat_charges_phys_therapy,inpat_charges_medsupps_chgd,inpat_charges_drgs_chgd,inpat_charges_emergency,inpat_charges_subtotal,outpat_charges_oper_room,outpat_charges_radiology_diag,outpat_charges_lab,outpat_charges_resp_therapy,outpat_charges_phys_therapy,outpat_charges_medsupps_chgd,outpat_charges_drgs_chgd,outpat_charges_emergency,outpat_charges_obs_beds,outpat_charges_subtotal,mdcr_outpat_charges,mdcr_outpat_costs,mdcr_inpat_costs,mdcr_inpat_charges_adpeds,mdcr_inpat_charges_anc_outp_oth,mdcr_outpat_lesser_costchg,gross_patient_rev_inpat_hosp,gross_patient_rev_inpat_anc,gross_patient_rev_inpat,gross_patient_rev_outpat,gross_patient_rev,allowances_discounts,net_patient_rev,operating_expenses,net_income_srvcs_patients,other_income_other,other_income_all,net_income_srvcs_patients_other,net_income,mdcd_charges,mdcd_costs,bed_days_available,mdcr_hmo_inpat_days,mdcr_inpat_days,mdcd_inpat_days,all_inpat_days,mdcr_inpat_discharges_ln1,mdcr_inpat_discharges,mdcd_inpat_discharges_ln1,mdcd_inpat_discharges,all_inpat_discharges_ln1,all_inpat_discharges,wt_cy,n_hosp_years_reporting,source_2552_10,cash_on_hand_and_in_banks,accounts_receivable,inventory,tot_cur_assets_genfund,tot_fixed_assets_genfund,tot_other_assets_genfund,tot_assets_genfund,tot_cur_liab_genfund,tot_longterm_liab_genfund,tot_liab_genfund,tot_fundbal_genfund,tot_liab_fundbal_genfund,cost_to_charge_ratio_for_uncomp,adult_peds_beds,tot_adult_peds_beds,beds,beds_incl_subprov,employees_FTEs,type_hosp,ownership_nonprofit,ownersh

In [99]:
display(pd.DataFrame(getPctMissing(redN_hospital_data_2018),columns=(["Null_2018%"])).rename_axis("variable").reset_index().head())
display(pd.DataFrame(getPctMissing(redN_hospital_data_2018),columns=(["Null_2019%"])).rename_axis("variable").reset_index().head())

,variable,Null_2018%
0,Medicare_provider_number,0.00
1,Hospital_name,0.03
2,State,0.00
3,"Hospital_Compare_5-star_rating_(October_2018,_...",0.00
4,Number_of_outpatient_services,0.00


,variable,Null_2019%
0,Medicare_provider_number,0.00
1,Hospital_name,0.03
2,State,0.00
3,"Hospital_Compare_5-star_rating_(October_2018,_...",0.00
4,Number_of_outpatient_services,0.00


### Data Classification

In [100]:
redN_hospital_data_2018["QualityStarCluster"]  = ['1Star' if x == 1 or x==2 else '2Star' if x==3 else '3Star' if x == 4 or x==5 else 'NULL' for x in redN_hospital_data_2018['Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)']] 
redN_hospital_data_2019["QualityStarCluster"]  = ['1Star' if x == 1 or x==2 else '2Star' if x==3 else '3Star' if x == 4 or x==5 else 'NULL' for x in redN_hospital_data_2019['Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)']]

In [101]:
p_class = redN_hospital_data_2018['Relative_price_for_outpatient_services'].describe(percentiles=[.3,.7]) #class=3
#display(p_class[[3,4,6,7]])
ls = []
for i, j in enumerate(redN_hospital_data_2018.loc[:,'Relative_price_for_outpatient_services']):
    if j <= p_class[4]: ls.append(1)
    elif p_class[4] < j <= p_class[6]: ls.append(2)
    elif j > p_class[6]: ls.append(3)
    else: ls.append(np.nan)
redN_hospital_data_2018.loc[:,'Class_3'] = ls
redN_hospital_data_2018 = redN_hospital_data_2018.sort_values('QualityStarCluster')
ls = []
for i, j in enumerate(redN_hospital_data_2019.loc[:,'Relative_price_for_outpatient_services']):
    if j <= p_class[4]: ls.append(1)
    elif p_class[4] < j <= p_class[6]: ls.append(2)
    elif j > p_class[6]: ls.append(3)
    else: ls.append(np.nan)
redN_hospital_data_2019.loc[:,'Class_3'] = ls
redN_hospital_data_2019 = redN_hospital_data_2019.sort_values('QualityStarCluster')

In [102]:
p_class = redN_hospital_data_2018['Relative_price_for_outpatient_services'].describe(percentiles=[.15,.35,.6,.85]) #class=5
#display(p_class[[3,4,5,7,8,9]])
ls = []
for i, j in enumerate(redN_hospital_data_2018.loc[:,'Relative_price_for_outpatient_services']):
    if j <= p_class[4]: ls.append(1)
    elif p_class[4] < j <= p_class[5]: ls.append(2)
    elif p_class[5] < j <= p_class[7]: ls.append(3)
    elif p_class[7] < j <= p_class[8]: ls.append(4)
    elif j > p_class[8]: ls.append(5)
    else: ls.append(np.nan)
redN_hospital_data_2018.loc[:,'Class_5'] = ls
redN_hospital_data_2018 = redN_hospital_data_2018.sort_values('QualityStarCluster')

ls = []
for i, j in enumerate(redN_hospital_data_2019.loc[:,'Relative_price_for_outpatient_services']):
    if j <= p_class[4]: ls.append(1)
    elif p_class[4] < j <= p_class[5]: ls.append(2)
    elif p_class[5] < j <= p_class[7]: ls.append(3)
    elif p_class[7] < j <= p_class[8]: ls.append(4)
    elif j > p_class[8]: ls.append(5)
    else: ls.append(np.nan)
redN_hospital_data_2019.loc[:,'Class_5'] = ls
redN_hospital_data_2019 = redN_hospital_data_2019.sort_values('QualityStarCluster')

In [103]:
p_class = redN_hospital_data_2018['Relative_price_for_outpatient_services'].describe(percentiles=[.15,.3,.5,.7,.85]) #class=6
#display(p_class[[3,4,5,6,7,8,9]])
ls = []
for i, j in enumerate(redN_hospital_data_2018.loc[:,'Relative_price_for_outpatient_services']):
    if j <= p_class[4]: ls.append(1)
    elif p_class[4] < j <= p_class[5]: ls.append(2)
    elif p_class[5] < j <= p_class[6]: ls.append(3)
    elif p_class[6] < j <= p_class[7]: ls.append(4)
    elif p_class[7] < j <= p_class[8]: ls.append(5)
    elif j > p_class[8]: ls.append(6)
    else: ls.append(np.nan)
redN_hospital_data_2018.loc[:,'Class_6'] = ls
redN_hospital_data_2018 = redN_hospital_data_2018.sort_values('QualityStarCluster')

ls = []
for i, j in enumerate(redN_hospital_data_2019.loc[:,'Relative_price_for_outpatient_services']):
    if j <= p_class[4]: ls.append(1)
    elif p_class[4] < j <= p_class[5]: ls.append(2)
    elif p_class[5] < j <= p_class[6]: ls.append(3)
    elif p_class[6] < j <= p_class[7]: ls.append(4)
    elif p_class[7] < j <= p_class[8]: ls.append(5)
    elif j > p_class[8]: ls.append(6)
    else: ls.append(np.nan)
redN_hospital_data_2019.loc[:,'Class_6'] = ls
redN_hospital_data_2019 = redN_hospital_data_2019.sort_values('QualityStarCluster')

In [104]:
p_class = redN_hospital_data_2018['Relative_price_for_outpatient_services'].describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9]) #class=10
#display(p_class[[3,4,5,6,7,8,9,10,11,12,13]])
ls = []
for i, j in enumerate(redN_hospital_data_2018.loc[:,'Relative_price_for_outpatient_services']):
    if j <= p_class[4]: ls.append(1)
    elif p_class[4] < j <= p_class[5]: ls.append(2)
    elif p_class[5] < j <= p_class[6]: ls.append(3)
    elif p_class[6] < j <= p_class[7]: ls.append(4)
    elif p_class[7] < j <= p_class[8]: ls.append(5)
    elif p_class[8] < j <= p_class[9]: ls.append(6)
    elif p_class[9] < j <= p_class[10]: ls.append(7)
    elif p_class[10] < j <= p_class[11]: ls.append(8)
    elif p_class[11] < j <= p_class[12]: ls.append(9)
    elif j > p_class[12]: ls.append(10)
    else: ls.append(np.nan)
redN_hospital_data_2018.loc[:,'Class_10'] = ls
redN_hospital_data_2018 = redN_hospital_data_2018.sort_values('QualityStarCluster')

ls = []
for i, j in enumerate(redN_hospital_data_2019.loc[:,'Relative_price_for_outpatient_services']):
    if j <= p_class[4]: ls.append(1)
    elif p_class[4] < j <= p_class[5]: ls.append(2)
    elif p_class[5] < j <= p_class[6]: ls.append(3)
    elif p_class[6] < j <= p_class[7]: ls.append(4)
    elif p_class[7] < j <= p_class[8]: ls.append(5)
    elif p_class[8] < j <= p_class[9]: ls.append(6)
    elif p_class[9] < j <= p_class[10]: ls.append(7)
    elif p_class[10] < j <= p_class[11]: ls.append(8)
    elif p_class[11] < j <= p_class[12]: ls.append(9)
    elif j > p_class[12]: ls.append(10)
    else: ls.append(np.nan)
redN_hospital_data_2019.loc[:,'Class_10'] = ls
redN_hospital_data_2019 = redN_hospital_data_2019.sort_values('QualityStarCluster')

In [105]:
#Creating a holdout set from random sample not part of training/validation
#These will be part of unseen scored predictions
holdout_sample = redN_hospital_data_2018.sample(100,random_state=1)[["Medicare_provider_number"]]
redN_hospital_data_2018 = redN_hospital_data_2018[~redN_hospital_data_2018['Medicare_provider_number'].isin(holdout_sample["Medicare_provider_number"])]
redN_hospital_data_2018["holdout_sample"] = 0
holdout_sample["holdout_sample"] = 1
redN_hospital_data_2019 = pd.merge(redN_hospital_data_2019, holdout_sample, on='Medicare_provider_number', how="left")
redN_hospital_data_2019 = redN_hospital_data_2019.fillna(0)
redN_hospital_data_2019 = redN_hospital_data_2019[redN_hospital_data_2019['Relative_price_for_outpatient_services']!=0]
#holdout_sample = pd.merge(holdout_sample, redN_hospital_data_2019, on='Medicare_provider_number', how="inner")
display(holdout_sample.head())

,Medicare_provider_number,holdout_sample
2266,390073,1
2581,450147,1
489,70008,1
1610,271325,1
2868,500041,1


In [106]:
print(redN_hospital_data_2018.shape)
display(redN_hospital_data_2018.head())
print(redN_hospital_data_2019.shape)
display(redN_hospital_data_2019.head())
print(holdout_sample.shape)
display(holdout_sample.head())

(2950, 310)


,Medicare_provider_number,Hospital_name,State,"Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)",Number_of_outpatient_services,Relative_price_for_outpatient_services,tot_expenses_employee_benefits,tot_expenses_admin_and_general,tot_expenses_housekeeping,tot_expenses_dietary,capasset_bldngsfixtrs_end10only,capasset_moveequip_end10only,capasset_subtotal_end10only,capasset_total_end10only,net_expenses_anc_dvc_chgd_only10,totcosts_devices_chgd,inpat_charges_devices_chgd,inpat_charges_total,outpat_charges_devices_chgd,outpat_charges_total,mdcd_net_revenue_only10,chgs_uninsured_charity_only10,costs_uninsured_charity_only10,uncomp_uninsured_charity_only10,baddebt_hospcmplx_only10,mdcr_baddebt_hospcmplx_only10,mdcr_baddebt_ln2701_only10,nonmdcr_baddebt_hospcmplx_only10,nonmdcr_baddebt_costs_only10,nonmdcr_uncomp_costs_only10,unreimb_uncomp_costs_only10,chgs_charity_patients_only10,costs_charity_patients_only10,uncomp_charity_patients_only10,mdcr_inpat_discharges_HMO_only10,expns_admin_and_general_salary,expns_nursing_admin_salary,expns_inp_adultspeds_salary,expns_anc_oper_room_salary,expns_anc_radiology_diag_salary,expns_anc_lab_salary,expns_anc_resp_therapy_salary,expns_outp_emergency_salary,expns_subtotal_salary,total_salaries,expenses_empeebens_other,expns_admin_and_general_other,expns_housekeeping_other,expns_dietary_other,total_other_expenses,capasset_bldngsfixtrs_beginning,capasset_moveequip_beginning,capasset_subtotal_beginning,capasset_total_beginning,capasset_moveequip_purchases,capasset_subtotal_purchases,capasset_total_purchases,net_expenses_cap_bldgs_fixtures,net_expenses_cap_movable_equip,net_expenses_employee_benefits,net_expenses_admin_and_general,net_expenses_operation_of_plant,net_expenses_laundry_and_linen,net_expenses_housekeeping,net_expenses_dietary,net_expenses_nursing_admin,net_expenses_medical_records,net_expenses_inp_adultspeds,net_expenses_anc_oper_room,net_expenses_anc_radiology_diag,net_expenses_anc_lab,net_expenses_anc_resp_therapy,net_expenses_anc_phys_therapy,net_expenses_anc_medsupps_chgd,net_expenses_anc_drgs_chgd,net_expenses_outp_emergency,net_expenses_subtotal,net_expenses_total,totcosts_adultspeds,totcosts_oper_room,totcosts_radiology_diag,totcosts_lab,totcosts_resp_therapy,totcosts_phys_therapy,totcosts_medsupps_chgd,totcosts_drgs_chgd,totcosts_emergency,totcosts_subtotal,totcosts_total,inpat_charges_adultspeds,inpat_charges_oper_room,inpat_charges_radiology_diag,inpat_charges_lab,inpat_charges_resp_therapy,inpat_charges_phys_therapy,inpat_charges_medsupps_chgd,inpat_charges_drgs_chgd,inpat_charges_emergency,inpat_charges_subtotal,outpat_charges_oper_room,outpat_charges_radiology_diag,outpat_charges_lab,outpat_charges_resp_therapy,outpat_charges_phys_therapy,outpat_charges_medsupps_chgd,outpat_charges_drgs_chgd,outpat_charges_emergency,outpat_charges_obs_beds,outpat_charges_subtotal,mdcr_outpat_charges,mdcr_outpat_costs,mdcr_inpat_costs,mdcr_inpat_charges_adpeds,mdcr_inpat_charges_anc_outp_oth,mdcr_outpat_lesser_costchg,gross_patient_rev_inpat_hosp,gross_patient_rev_inpat_anc,gross_patient_rev_inpat,gross_patient_rev_outpat,gross_patient_rev,allowances_discounts,net_patient_rev,operating_expenses,net_income_srvcs_patients,other_income_other,other_income_all,net_income_srvcs_patients_other,net_income,mdcd_charges,mdcd_costs,bed_days_available,mdcr_hmo_inpat_days,mdcr_inpat_days,mdcd_inpat_days,all_inpat_days,mdcr_inpat_discharges_ln1,mdcr_inpat_discharges,mdcd_inpat_discharges_ln1,mdcd_inpat_discharges,all_inpat_discharges_ln1,all_inpat_discharges,wt_cy,n_hosp_years_reporting,source_2552_10,cash_on_hand_and_in_banks,accounts_receivable,inventory,tot_cur_assets_genfund,tot_fixed_assets_genfund,tot_other_assets_genfund,tot_assets_genfund,tot_cur_liab_genfund,tot_longterm_liab_genfund,tot_liab_genfund,tot_fundbal_genfund,tot_liab_fundbal_genfund,cost_to_charge_ratio_for_uncomp,adult_peds_beds,tot_adult_peds_beds,beds,beds_incl_subprov,employees_FTEs,type_hosp,ownership_nonprofit,ownersh

(3004, 310)


,Medicare_provider_number,Hospital_name,State,"Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)",Number_of_outpatient_services,Relative_price_for_outpatient_services,tot_expenses_employee_benefits,tot_expenses_admin_and_general,tot_expenses_housekeeping,tot_expenses_dietary,capasset_bldngsfixtrs_end10only,capasset_moveequip_end10only,capasset_subtotal_end10only,capasset_total_end10only,net_expenses_anc_dvc_chgd_only10,totcosts_devices_chgd,inpat_charges_devices_chgd,inpat_charges_total,outpat_charges_devices_chgd,outpat_charges_total,mdcd_net_revenue_only10,chgs_uninsured_charity_only10,costs_uninsured_charity_only10,uncomp_uninsured_charity_only10,baddebt_hospcmplx_only10,mdcr_baddebt_hospcmplx_only10,mdcr_baddebt_ln2701_only10,nonmdcr_baddebt_hospcmplx_only10,nonmdcr_baddebt_costs_only10,nonmdcr_uncomp_costs_only10,unreimb_uncomp_costs_only10,chgs_charity_patients_only10,costs_charity_patients_only10,uncomp_charity_patients_only10,mdcr_inpat_discharges_HMO_only10,expns_admin_and_general_salary,expns_nursing_admin_salary,expns_inp_adultspeds_salary,expns_anc_oper_room_salary,expns_anc_radiology_diag_salary,expns_anc_lab_salary,expns_anc_resp_therapy_salary,expns_outp_emergency_salary,expns_subtotal_salary,total_salaries,expenses_empeebens_other,expns_admin_and_general_other,expns_housekeeping_other,expns_dietary_other,total_other_expenses,capasset_bldngsfixtrs_beginning,capasset_moveequip_beginning,capasset_subtotal_beginning,capasset_total_beginning,capasset_moveequip_purchases,capasset_subtotal_purchases,capasset_total_purchases,net_expenses_cap_bldgs_fixtures,net_expenses_cap_movable_equip,net_expenses_employee_benefits,net_expenses_admin_and_general,net_expenses_operation_of_plant,net_expenses_laundry_and_linen,net_expenses_housekeeping,net_expenses_dietary,net_expenses_nursing_admin,net_expenses_medical_records,net_expenses_inp_adultspeds,net_expenses_anc_oper_room,net_expenses_anc_radiology_diag,net_expenses_anc_lab,net_expenses_anc_resp_therapy,net_expenses_anc_phys_therapy,net_expenses_anc_medsupps_chgd,net_expenses_anc_drgs_chgd,net_expenses_outp_emergency,net_expenses_subtotal,net_expenses_total,totcosts_adultspeds,totcosts_oper_room,totcosts_radiology_diag,totcosts_lab,totcosts_resp_therapy,totcosts_phys_therapy,totcosts_medsupps_chgd,totcosts_drgs_chgd,totcosts_emergency,totcosts_subtotal,totcosts_total,inpat_charges_adultspeds,inpat_charges_oper_room,inpat_charges_radiology_diag,inpat_charges_lab,inpat_charges_resp_therapy,inpat_charges_phys_therapy,inpat_charges_medsupps_chgd,inpat_charges_drgs_chgd,inpat_charges_emergency,inpat_charges_subtotal,outpat_charges_oper_room,outpat_charges_radiology_diag,outpat_charges_lab,outpat_charges_resp_therapy,outpat_charges_phys_therapy,outpat_charges_medsupps_chgd,outpat_charges_drgs_chgd,outpat_charges_emergency,outpat_charges_obs_beds,outpat_charges_subtotal,mdcr_outpat_charges,mdcr_outpat_costs,mdcr_inpat_costs,mdcr_inpat_charges_adpeds,mdcr_inpat_charges_anc_outp_oth,mdcr_outpat_lesser_costchg,gross_patient_rev_inpat_hosp,gross_patient_rev_inpat_anc,gross_patient_rev_inpat,gross_patient_rev_outpat,gross_patient_rev,allowances_discounts,net_patient_rev,operating_expenses,net_income_srvcs_patients,other_income_other,other_income_all,net_income_srvcs_patients_other,net_income,mdcd_charges,mdcd_costs,bed_days_available,mdcr_hmo_inpat_days,mdcr_inpat_days,mdcd_inpat_days,all_inpat_days,mdcr_inpat_discharges_ln1,mdcr_inpat_discharges,mdcd_inpat_discharges_ln1,mdcd_inpat_discharges,all_inpat_discharges_ln1,all_inpat_discharges,wt_cy,n_hosp_years_reporting,source_2552_10,cash_on_hand_and_in_banks,accounts_receivable,inventory,tot_cur_assets_genfund,tot_fixed_assets_genfund,tot_other_assets_genfund,tot_assets_genfund,tot_cur_liab_genfund,tot_longterm_liab_genfund,tot_liab_genfund,tot_fundbal_genfund,tot_liab_fundbal_genfund,cost_to_charge_ratio_for_uncomp,adult_peds_beds,tot_adult_peds_beds,beds,beds_incl_subprov,employees_FTEs,type_hosp,ownership_nonprofit,ownersh

(100, 2)


,Medicare_provider_number,holdout_sample
2266,390073,1
2581,450147,1
489,70008,1
1610,271325,1
2868,500041,1


In [107]:
holdout_sample.to_csv("holdout_sample.csv",index=False)
redN_hospital_data_2018.to_csv("rand_"+Training_rand+".csv",index=False)
redN_hospital_data_2019.to_csv("rand_"+Scoring_rand+".csv",index=False)